In [67]:
from drlqap.taskgenerators import generators
import scipy.optimize
from drlqap.simplesolver import solve_qap_backtracking, solve_qap_maxgreedy, solve_qap_faq, solve_random
from drlqap.gurobi import solve_qap_gurobi
import numpy as np
from drlqap.qap import QAP

In [2]:
# change to project root
%cd ..

/home/jupyter-tim/ba-tim


In [3]:
generators

{'minilinear': <drlqap.taskgenerators.LinearTaskGenerator at 0x7f55d1d0b250>,
 'small_random_graphs': <drlqap.taskgenerators.RandomWeightsTaskGenerator at 0x7f55d1d0b310>,
 'medium_random_graphs': <drlqap.taskgenerators.RandomWeightsTaskGenerator at 0x7f55d1d0b3d0>,
 'qaplib_bur26a': <drlqap.taskgenerators.LazyGlobTaskGenerator at 0x7f55d1d0b490>,
 'qaplib_bur26a_normalized': <drlqap.taskgenerators.LazyGlobTaskGenerator at 0x7f55d1d0b4f0>,
 'small_fixed': <drlqap.taskgenerators.LazyGlobTaskGenerator at 0x7f55d1d0b580>,
 'triangle': <drlqap.taskgenerators.SingleTask at 0x7f55d1d0b610>,
 'qaplib_all_bur': <drlqap.taskgenerators.LazyGlobTaskGenerator at 0x7f55d1d0b700>,
 'qaplib_sko_42_64_normalized': <drlqap.taskgenerators.LazyGlobTaskGenerator at 0x7f55d1d0b6a0>,
 'qaplib_all_bur_normalized': <drlqap.taskgenerators.LazyGlobTaskGenerator at 0x7f55d1d0b7c0>,
 'qaplib_all_chr_normalized': <drlqap.taskgenerators.LazyGlobTaskGenerator at 0x7f55d1d0b820>,
 'qaplib_all_esc_normalized': <drlqap

In [4]:
qap = generators['medium_random_graphs'].sample()

In [5]:
solve_qap_maxgreedy(qap)

(51.695816457271576, [0, 2, 3, 10, 6, 1, 13, 7, 12, 9, 11, 15, 4, 5, 14, 8])

In [6]:
solve_qap_faq(qap)

(47.00905,
 array([10,  4,  0, 15,  2, 11, 13, 12,  6,  1,  7, 14,  9,  5,  3,  8]))

In [7]:
#solve_qap_gurobi(qap)

In [8]:
%timeit solve_qap_faq(qap)

3.58 ms ± 30.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
!ls runs

a2c_mediumrandoms			 dqn_dense_ms_ec_eps0_mini
a2c_ms100x_mediumrandoms		 dqn_dense_ms_ec_eps0_rni_study
a2c_ms100x_mediumrandoms_cyclic		 dqn_dense_ms_ec_eps0_smallrandoms
a2c_ms100x_mediumrandoms_cyclic_wronglr  dqn_dense_ms_ec_mediumrandoms
a2c_ms100x_mediumrandoms_stepped	 dqn_linkedqap_smallrandoms
dqn_dense_ec_eps0_norm_study		 reinforce_ms100x_mediumrandoms
dqn_dense_ec_eps0_norm_study_2		 reinforce_ms100x_smallrandoms
dqn_dense_ms_ec_eps0_mediumrandoms


In [10]:
!ls runs/dqn_dense_ms_ec_eps0_smallrandoms

lr1e-4_s1  lr1e-4_s4  lr1e-5_s3  lr3e-4_s2  lr5e-4_s1  lr5e-4_s4
lr1e-4_s2  lr1e-5_s1  lr1e-5_s4  lr3e-4_s3  lr5e-4_s2
lr1e-4_s3  lr1e-5_s2  lr3e-4_s1  lr3e-4_s4  lr5e-4_s3


In [11]:
from drlqap.evaltools import load_checkpoints
from pathlib import Path

In [12]:
agents = load_checkpoints(Path('runs/a2c_ms100x_mediumrandoms/lr4e-5_s3'))

runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_0.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_1000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_2000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_3000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_4000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_5000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_6000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_7000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_8000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_9000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_10000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_11000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_12000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_13000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_14000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_15000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_

In [13]:
agents[-1].solve(qap)

(51.85995373129845, [11, 6, 7, 1, 10, 2, 15, 5, 3, 0, 12, 9, 4, 14, 13, 8])

## Average results

In [39]:
def evaluate(solver, problem, samples):
    values = []
    for i in range(samples):
        qap = generators[problem].sample()
        v_solver, assignment = solver(qap)
        v = qap.compute_value(assignment)
        if not np.isclose(v, v_solver):
            print (f"solver outputs incorrect value (got {v_solver}, actual {v})")
        values.append(v)
    return np.mean(values)

def evaluate_set(solver, problem):
    values = []
    test_set = generators[problem].test_set()
    for qap in test_set:
        v_solver, assignment = solver(qap)
        v = qap.compute_value(assignment)
        if not np.isclose(v, v_solver):
            print (f"solver outputs incorrect value (got {v_solver}, actual {v})")
        values.append(v)
    return np.mean(values)

In [40]:
evaluate(solve_qap_faq, 'small_random_graphs', 1000)

11.439265

In [41]:
evaluate(agents[-1].solve, 'small_random_graphs', 500)

11.591277

In [42]:
evaluate(solve_qap_faq, 'medium_random_graphs', 1000)

49.857094

In [43]:
evaluate(agents[-1].solve, 'medium_random_graphs', 500)

52.509396

In [44]:
evaluate_set(solve_qap_faq, 'qaplib_all_64_normalized')

solver outputs incorrect value (got 5417189.0, actual 5542958.0)
solver outputs incorrect value (got 3797940.0, actual 3885741.0)
solver outputs incorrect value (got 5418795.0, actual 5565287.0)
solver outputs incorrect value (got 3861501.0, actual 3963769.0)
solver outputs incorrect value (got 5338460.0, actual 5489722.0)
solver outputs incorrect value (got 3782935.0, actual 3888533.0)
solver outputs incorrect value (got 9688100.0, actual 10297388.0)
solver outputs incorrect value (got 7239618.0, actual 7664970.0)


31941316.0

In [45]:
evaluate_set(agents[-1].solve, 'qaplib_all_64_normalized')

55563884.0

In [46]:
evaluate_set(solve_qap_faq, 'qaplib_tai35a_normalized')

2514002.0

In [47]:
evaluate_set(agents[-1].solve, 'qaplib_tai35a_normalized')

2755434.0

In [48]:
evaluate_set(agents[-1].solve, 'qaplib_tai35a_normalized')

2755434.0

## Qaplib

In [49]:
qaplib = generators["qaplib_all_64_normalized"].test_set()
qaplib_by_name = {q.name: q for q in qaplib}

In [50]:
agents[-1].solve(qaplib_by_name["bur26c"])

(6153352.0,
 [0,
  19,
  2,
  13,
  18,
  23,
  21,
  25,
  10,
  24,
  14,
  4,
  15,
  8,
  7,
  22,
  6,
  3,
  5,
  20,
  12,
  1,
  17,
  11,
  16,
  9])

# Asymmetry

In [52]:
def mirror(qap):
    a = qap.A
    b = qap.B
    return QAP(b, a, qap.linear_costs.transpose(0,1), 0)
    
def mirror_assignment(assignment):
    out = [None] * len(assignment)
    for i, j in enumerate(assignment):
        out[j] = i
    return out

In [53]:
qap_mirrored = mirror(qap)

In [54]:
qap.compute_value([12, 13, 2, 10, 1, 14, 4, 7, 6, 8, 3, 11, 9, 0, 5, 15])

tensor(56.0204)

In [55]:
qap_mirrored.compute_value(mirror_assignment([12, 13, 2, 10, 1, 14, 4, 7, 6, 8, 3, 11, 9, 0, 5, 15]))

tensor(56.0204)

In [56]:
agents[-1].solve(qap)

(51.85995373129845, [11, 6, 7, 1, 10, 2, 15, 5, 3, 0, 12, 9, 4, 14, 13, 8])

In [65]:
v, ass = agents[-1].solve(qap_mirrored)
print(v, ass, qap_mirrored.compute_value(ass))

50.36963647603989 [11, 9, 5, 14, 3, 13, 2, 7, 4, 10, 6, 1, 0, 12, 8, 15] tensor(50.3696)


In [58]:
evaluate_set(lambda qap: solve_qap_faq(mirror(qap)), 'qaplib_all_64_normalized')

solver outputs incorrect value (got 5370098.0, actual 5764143.0)
solver outputs incorrect value (got 3766727.0, actual 4289858.0)
solver outputs incorrect value (got 5474199.0, actual 6011081.0)
solver outputs incorrect value (got 3851803.0, actual 4235187.0)
solver outputs incorrect value (got 5385812.0, actual 5912032.0)
solver outputs incorrect value (got 3753417.0, actual 4236290.0)
solver outputs incorrect value (got 9676538.0, actual 11076894.0)
solver outputs incorrect value (got 6731962.0, actual 7874443.0)
solver outputs incorrect value (got 33082.0, actual 55664.0)
solver outputs incorrect value (got 10468.0, actual 39730.0)
solver outputs incorrect value (got 13088.0, actual 23448.0)
solver outputs incorrect value (got 16398.0, actual 66120.0)
solver outputs incorrect value (got 9112.0, actual 61150.0)
solver outputs incorrect value (got 11936.0, actual 54186.0)
solver outputs incorrect value (got 15440.0, actual 96236.0)
solver outputs incorrect value (got 2152.0, actual 39

55092540.0

In [59]:
evaluate_set(lambda qap: agents[-1].solve(mirror(qap)), 'qaplib_all_64_normalized')

solver outputs incorrect value (got 5745448.0, actual 5858658.0)
solver outputs incorrect value (got 3991486.0, actual 4161468.0)
solver outputs incorrect value (got 5657299.0, actual 5853167.0)
solver outputs incorrect value (got 4003360.0, actual 4216700.0)
solver outputs incorrect value (got 5725134.0, actual 5974355.0)
solver outputs incorrect value (got 4027679.0, actual 4116851.0)
solver outputs incorrect value (got 10899164.0, actual 11412922.0)
solver outputs incorrect value (got 7624172.0, actual 7613862.0)
solver outputs incorrect value (got 15432.0, actual 56490.0)
solver outputs incorrect value (got 27320.0, actual 47774.0)
solver outputs incorrect value (got 25496.0, actual 37966.0)
solver outputs incorrect value (got 28998.0, actual 50952.0)
solver outputs incorrect value (got 41718.0, actual 75924.0)
solver outputs incorrect value (got 29290.0, actual 64786.0)
solver outputs incorrect value (got 52546.0, actual 70278.0)
solver outputs incorrect value (got 3918.0, actual 

59285356.0

In [60]:
evaluate(lambda qap: agents[-1].solve(mirror(qap)), 'medium_random_graphs', 500)

solver outputs incorrect value (got 62.123902440071106, actual 68.03571319580078)
solver outputs incorrect value (got 46.510784359648824, actual 52.41948699951172)
solver outputs incorrect value (got 57.94299528375268, actual 63.86463165283203)
solver outputs incorrect value (got 60.15311360359192, actual 70.80875396728516)
solver outputs incorrect value (got 54.130779184401035, actual 57.546592712402344)
solver outputs incorrect value (got 46.388225331902504, actual 53.52107620239258)
solver outputs incorrect value (got 52.49503681063652, actual 55.522926330566406)
solver outputs incorrect value (got 55.61092674732208, actual 61.49651336669922)
solver outputs incorrect value (got 54.242419958114624, actual 62.84373092651367)
solver outputs incorrect value (got 51.938814252614975, actual 60.2183952331543)
solver outputs incorrect value (got 49.343894615769386, actual 57.36058044433594)
solver outputs incorrect value (got 55.91042836755514, actual 64.48477172851562)
solver outputs incor

solver outputs incorrect value (got 54.91096958518028, actual 59.84571838378906)
solver outputs incorrect value (got 49.75399029254913, actual 58.663639068603516)
solver outputs incorrect value (got 53.14172950387001, actual 58.13994598388672)
solver outputs incorrect value (got 55.34204247780144, actual 58.63999557495117)
solver outputs incorrect value (got 53.861756309866905, actual 59.670562744140625)
solver outputs incorrect value (got 57.7673060297966, actual 61.61551284790039)
solver outputs incorrect value (got 63.53135305643082, actual 68.01343536376953)
solver outputs incorrect value (got 54.186367869377136, actual 64.91064453125)
solver outputs incorrect value (got 51.47192317247391, actual 58.98811340332031)
solver outputs incorrect value (got 61.41011949442327, actual 67.49482727050781)
solver outputs incorrect value (got 46.536518052220345, actual 51.850528717041016)
solver outputs incorrect value (got 55.10616374015808, actual 57.216148376464844)
solver outputs incorrect 

solver outputs incorrect value (got 46.19947502017021, actual 52.54985046386719)
solver outputs incorrect value (got 52.35625907545909, actual 62.23399353027344)
solver outputs incorrect value (got 53.04157914966345, actual 59.15952682495117)
solver outputs incorrect value (got 60.45163545012474, actual 69.4272232055664)
solver outputs incorrect value (got 37.35365053266287, actual 46.37726974487305)
solver outputs incorrect value (got 57.31477749347687, actual 60.9365348815918)
solver outputs incorrect value (got 47.61482137441635, actual 55.0692138671875)
solver outputs incorrect value (got 56.088831067085266, actual 61.386260986328125)
solver outputs incorrect value (got 53.19412988424301, actual 62.393009185791016)
solver outputs incorrect value (got 43.902492955327034, actual 50.91974639892578)
solver outputs incorrect value (got 46.74099922180176, actual 53.8012809753418)
solver outputs incorrect value (got 56.66469073295593, actual 63.92325973510742)
solver outputs incorrect val

solver outputs incorrect value (got 57.75276492815465, actual 62.81904983520508)
solver outputs incorrect value (got 45.933306127786636, actual 51.5599365234375)
solver outputs incorrect value (got 48.734740652143955, actual 55.182430267333984)
solver outputs incorrect value (got 48.96376818418503, actual 59.665061950683594)
solver outputs incorrect value (got 47.1704124212265, actual 56.05052185058594)
solver outputs incorrect value (got 52.7172589302063, actual 62.82904815673828)
solver outputs incorrect value (got 52.187460243701935, actual 59.76036071777344)
solver outputs incorrect value (got 58.35999542474747, actual 63.93376922607422)
solver outputs incorrect value (got 51.26465791463852, actual 57.95883560180664)
solver outputs incorrect value (got 53.56761712580919, actual 58.706443786621094)
solver outputs incorrect value (got 52.03050947189331, actual 58.967132568359375)
solver outputs incorrect value (got 56.64494113624096, actual 62.89456558227539)
solver outputs incorrect

solver outputs incorrect value (got 50.83274146914482, actual 57.96160125732422)
solver outputs incorrect value (got 53.21504789282335, actual 62.4484977722168)
solver outputs incorrect value (got 55.38301455974579, actual 63.37984085083008)
solver outputs incorrect value (got 50.97127765417099, actual 61.00724411010742)
solver outputs incorrect value (got 39.99444341659546, actual 50.23088073730469)
solver outputs incorrect value (got 55.45690280199051, actual 58.033668518066406)
solver outputs incorrect value (got 44.67476287484169, actual 54.711238861083984)
solver outputs incorrect value (got 47.94948276877403, actual 54.124515533447266)
solver outputs incorrect value (got 46.11927714943886, actual 49.19157409667969)
solver outputs incorrect value (got 60.844832330942154, actual 66.42032623291016)
solver outputs incorrect value (got 58.235127389431, actual 61.75398635864258)
solver outputs incorrect value (got 57.47826348245144, actual 63.53126907348633)
solver outputs incorrect va

59.986713

In [61]:
evaluate(lambda qap: agents[-1].solve(mirror(qap)), 'small_random_graphs', 500)

solver outputs incorrect value (got 12.829694986343384, actual 15.990316390991211)
solver outputs incorrect value (got 14.958448231220245, actual 17.397720336914062)
solver outputs incorrect value (got 12.224725686013699, actual 12.743021011352539)
solver outputs incorrect value (got 13.535929948091507, actual 16.144622802734375)
solver outputs incorrect value (got 14.924028992652893, actual 15.894733428955078)
solver outputs incorrect value (got 11.950022898614407, actual 12.804084777832031)
solver outputs incorrect value (got 11.009913004934788, actual 12.794928550720215)
solver outputs incorrect value (got 11.443582870066166, actual 13.168905258178711)
solver outputs incorrect value (got 9.812325403094292, actual 12.278125762939453)
solver outputs incorrect value (got 13.295435577630997, actual 15.093351364135742)
solver outputs incorrect value (got 10.739628911018372, actual 12.795788764953613)
solver outputs incorrect value (got 10.778914749622345, actual 14.402247428894043)
solve

solver outputs incorrect value (got 9.90972200781107, actual 11.580865859985352)
solver outputs incorrect value (got 10.187393590807915, actual 10.474275588989258)
solver outputs incorrect value (got 13.00329178571701, actual 15.098844528198242)
solver outputs incorrect value (got 16.67906880378723, actual 17.853309631347656)
solver outputs incorrect value (got 9.25515810586512, actual 11.860898971557617)
solver outputs incorrect value (got 9.904480755329132, actual 10.893322944641113)
solver outputs incorrect value (got 10.964590728282928, actual 14.57613754272461)
solver outputs incorrect value (got 10.539275586605072, actual 13.3040132522583)
solver outputs incorrect value (got 17.36543995141983, actual 19.58455467224121)
solver outputs incorrect value (got 13.459457740187645, actual 14.926559448242188)
solver outputs incorrect value (got 12.392851054668427, actual 14.759224891662598)
solver outputs incorrect value (got 13.361527025699615, actual 16.237905502319336)
solver outputs i

solver outputs incorrect value (got 11.728026330471039, actual 12.843334197998047)
solver outputs incorrect value (got 12.462341159582138, actual 14.925896644592285)
solver outputs incorrect value (got 7.984726779162884, actual 11.914514541625977)
solver outputs incorrect value (got 11.593078196048737, actual 14.87825870513916)
solver outputs incorrect value (got 8.025844492018223, actual 12.33630084991455)
solver outputs incorrect value (got 12.0622149258852, actual 13.692827224731445)
solver outputs incorrect value (got 16.116718888282776, actual 18.074031829833984)
solver outputs incorrect value (got 12.203712165355682, actual 14.976927757263184)
solver outputs incorrect value (got 15.31250113248825, actual 15.883572578430176)
solver outputs incorrect value (got 9.85312204156071, actual 13.393867492675781)
solver outputs incorrect value (got 10.40248292684555, actual 13.360122680664062)
solver outputs incorrect value (got 10.481021404266357, actual 12.84471607208252)
solver outputs 

solver outputs incorrect value (got 8.129860758781433, actual 9.606121063232422)
solver outputs incorrect value (got 12.998889550566673, actual 15.177227020263672)
solver outputs incorrect value (got 10.930524110794067, actual 12.812888145446777)
solver outputs incorrect value (got 14.880101785063744, actual 15.924683570861816)
solver outputs incorrect value (got 13.950215220451355, actual 17.191783905029297)
solver outputs incorrect value (got 15.280255060642958, actual 16.89116096496582)
solver outputs incorrect value (got 9.307730883359909, actual 11.299582481384277)
solver outputs incorrect value (got 15.775642551481724, actual 17.590604782104492)
solver outputs incorrect value (got 13.527632892131805, actual 15.67641544342041)
solver outputs incorrect value (got 13.625615417957306, actual 15.290472030639648)
solver outputs incorrect value (got 10.026324450969696, actual 13.560014724731445)
solver outputs incorrect value (got 10.613898158073425, actual 11.726999282836914)
solver ou

solver outputs incorrect value (got 10.86144107580185, actual 11.212926864624023)
solver outputs incorrect value (got 8.030446320772171, actual 11.803950309753418)
solver outputs incorrect value (got 10.782724276185036, actual 12.898003578186035)
solver outputs incorrect value (got 12.91729211807251, actual 14.207369804382324)
solver outputs incorrect value (got 8.461907595396042, actual 11.696215629577637)
solver outputs incorrect value (got 13.756340535357594, actual 16.887134552001953)
solver outputs incorrect value (got 12.3535957634449, actual 14.923394203186035)
solver outputs incorrect value (got 15.100824892520905, actual 17.109838485717773)
solver outputs incorrect value (got 12.553482204675674, actual 15.471911430358887)
solver outputs incorrect value (got 10.955697566270828, actual 11.679359436035156)
solver outputs incorrect value (got 7.933312300592661, actual 10.867301940917969)
solver outputs incorrect value (got 7.51042303442955, actual 10.348873138427734)
solver output

13.905738

In [62]:
agents[-1].solve(mirror(qaplib_by_name["bur26c"]))

(5657299.0,
 [12,
  15,
  1,
  14,
  18,
  5,
  2,
  4,
  11,
  24,
  20,
  13,
  3,
  10,
  22,
  8,
  23,
  16,
  9,
  17,
  7,
  6,
  0,
  19,
  21,
  25])

In [ ]:
evaluate_set(agents[-1].solve, 'qaplib_all_bur_normalized')

In [ ]:
evaluate_set(lambda qap: agents[-1].solve(mirror(qap)), 'qaplib_all_bur_normalized')

# 